In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-27T12:11:43.899667+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')

In [3]:
# Get the token from Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVELOOP_ORG_ID

In [4]:
import langchain
import newspaper
import requests
import deeplake
%watermark --iversions 

requests : 2.31.0
deeplake : 3.8.6
newspaper: 0.2.8
langchain: 0.0.336



In [5]:
# We scrape several Artificial Intelligence news

import requests
from newspaper import Article # https://github.com/codelucas/newspaper
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/"
]

session = requests.Session()
pages_content = [] # where we save the scraped articles

for url in article_urls:
    try:
        time.sleep(2) # sleep two seconds for gentle scraping
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download() # download HTML of webpage
            article.parse() # parse HTML to extract the article text
            pages_content.append({ "url": url, "text": article.text })
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

#If an error occurs while fetching an article, we catch the exception and print
#an error message. This ensures that even if one article fails to download,
#the rest of the articles can still be processed.

Then, we import the OpenAIEmbeddings class, which will be used to compute embeddings for our documents. We also import the Deep Lake class from the langchain.vectorstores module will serve as the storage for our documents and their embeddings. 

By setting up the Deep Lake instance with a specified dataset path and the embedding_function parameter set to the OpenAIEmbeddings instance, we establish a connection to Deep Lake and configure it to use the specified embedding model for computing document embeddings. 

In [6]:
# We'll use an embedding model to compute our documents' embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# We'll store the documents and their embeddings in the deep lake vector db
from langchain.vectorstores import DeepLake

# Setup deep lake
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

my_activeloop_dataset_name = "langchain_course_analysis_outline"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Your Deep Lake dataset has been successfully created!


Next, we create an instance of RecursiveCharacterTextSplitter with specified chunk_size and chunk_overlap parameters. Then, we iterated over the pages_content and use the split_text method of the text_splitter to split each article text into chunks. These chunks are then appended to the all_texts list, resulting in a collection of smaller text chunks derived from the original articles. 

In [7]:
# We split the article texts into small chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts = []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        all_texts.append(chunk)

Now we can add those chunks to the Deep Lake database.

In [8]:
# we add all the chunks to the Deep lake
db.add_texts(all_texts)

Creating 94 embeddings in 1 batches of size 94:: 100%|███████████████████████████████████| 1/1 [00:45<00:00, 45.02s/it]

Dataset(path='hub://jfrancis/langchain_course_analysis_outline', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (94, 1)      str     None   
 metadata     json      (94, 1)      str     None   
 embedding  embedding  (94, 1536)  float32   None   
    id        text      (94, 1)      str     None   


['e7b21203-8cf0-11ee-b60b-401c83da435e',
 'e7b21204-8cf0-11ee-98a7-401c83da435e',
 'e7b21205-8cf0-11ee-b082-401c83da435e',
 'e7b21206-8cf0-11ee-b8f9-401c83da435e',
 'e7b21207-8cf0-11ee-9aea-401c83da435e',
 'e7b21208-8cf0-11ee-8297-401c83da435e',
 'e7b21209-8cf0-11ee-9092-401c83da435e',
 'e7b2120a-8cf0-11ee-bdca-401c83da435e',
 'e7b2120b-8cf0-11ee-a0b8-401c83da435e',
 'e7b2120c-8cf0-11ee-9f19-401c83da435e',
 'e7b2120d-8cf0-11ee-99ea-401c83da435e',
 'e7b2120e-8cf0-11ee-bac4-401c83da435e',
 'e7b2120f-8cf0-11ee-b229-401c83da435e',
 'e7b21210-8cf0-11ee-b51c-401c83da435e',
 'e7b21211-8cf0-11ee-9369-401c83da435e',
 'e7b21212-8cf0-11ee-bdf0-401c83da435e',
 'e7b21213-8cf0-11ee-9e80-401c83da435e',
 'e7b21214-8cf0-11ee-96ea-401c83da435e',
 'e7b21215-8cf0-11ee-a556-401c83da435e',
 'e7b21216-8cf0-11ee-b910-401c83da435e',
 'e7b21217-8cf0-11ee-ba1e-401c83da435e',
 'e7b21218-8cf0-11ee-8dfa-401c83da435e',
 'e7b21219-8cf0-11ee-a04c-401c83da435e',
 'e7b2121a-8cf0-11ee-88e9-401c83da435e',
 'e7b2121b-8cf0-

Since the dataset is stored on Activeloop, you can load it later without recomputing embeddings. This is a significant benefit cause it would save you time and computational resources. LangChain has a wrapper around Deep Lake, allowing you to use it as a Vector Store.

In [9]:
#db = DeepLake(dataset_path=dataset_path,read_only=True,embedding=embeddings)

We are done with setting up the Deep Lake dataset with our documents! Let’s now focus on building the “Plan and Execute” agent that will leverage our dataset. Now, we can set up our Plan and Execute agent. Let’s create a retriever from the Deep Lake dataset and a function for our custom tool that retrieves the most similar documents to a query from the dataset.

In [9]:
# Get the retriever object from the deep lake db object and set the number
# of retrieved documents to 3
retriever = db.as_retriever()
retriever.search_kwargs['k'] = 3

# We define some variables that will be used inside our custom tool
CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n" # how to join together the retrieved docs to form a single string

# This is the function that defines our custom tool that retrieves relevant
# docs from Deep Lake
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------"
    return texts_merged

We retrieve the retriever object from the Deep Lake database and set the number of retrieved documents to 3. This is important for the plan and execution agent because it allows us to retrieve a specific number of relevant documents from Deep Lake based on a given query.

Also, we defined a custom tool function called retrieve_n_docs_tool that takes a query as input and uses the retriever to search for relevant documents containing the answer to the query. 

The retrieved document texts are then merged using the CUSTOM_TOOL_DOCS_SEPARATOR variable, representing the separator string used to join the documents into a single string. The merged text is returned as the output of the custom tool function. 

This functionality enables the plan and execution agent to retrieve and process relevant documents for further analysis and decision-making.

In [10]:
from langchain.agents.tools import Tool

# We create the tool that uses the "retrieve_n_docs_tool" function
tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]

The tool is named "Search Private Docs," and its functionality is based on the retrieve_n_docs_tool function. The purpose of this tool is to provide a way to search for and retrieve relevant documents from Deep Lake in order to answer questions about current events related to Artificial Intelligence. The tool is described as being useful in situations where there is a need to gather information and insights from private documents. 

We are now ready to create the agent!

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

# let's create the Plan and Execute agent
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

The agent consists of two components: a planner and an executor. The planner is responsible for generating a plan based on the given input, and the executor executes the plan by interacting with the tools and external systems. The agent is set to be verbose, which means it will provide detailed information and logs during its operation.

In [13]:
# we test the agent
response = agent.run("Write an overview of Artificial Intelligence regulations by governments by country")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research and gather information on the current state of Artificial Intelligence (AI) regulations by governments in different countries.'), Step(value='Organize the information by country and categorize it based on the level of AI regulation.'), Step(value='Provide an overview of the AI regulations in each country, including any specific laws or policies that have been implemented.'), Step(value='Include information on the key areas covered by the regulations, such as data privacy, algorithmic transparency, liability, and ethical considerations.'), Step(value='Highlight any notable differences or similarities between the regulations in different countries.'), Step(value='Summarize the overall trends and developments in AI regulations globally.'), Step(value='Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country. ')]

> Entering new AgentExecutor chain...
Action:
```
{


Thought:Based on the information gathered from the search, here is the organized information by country and the level of AI regulation:

1. European Union (EU):
   - The EU has approved the AI Act, which categorizes AI systems based on potential harm.
   - The AI Act introduces transparency and risk-management rules for AI systems.
   - Exemptions are provided for research activities and AI components under open-source licenses.
   - Regulatory sandboxes are promoted to test AI before deployment.
   - Citizens have the right to file complaints about AI systems and receive explanations of decisions.
   - The EU AI Office will monitor the implementation of the AI rulebook.

2. United States (US):
   - The US does not have comprehensive federal AI regulation in place yet.
   - The US takes an experimental approach to AI development.
   - Market and product fit is established first, followed by retrofitting to regulatory frameworks.
   - This approach fosters innovation among US-based AI d

Thought:The search in the private documents did not provide any relevant information on the key areas covered by AI regulations, such as data privacy, algorithmic transparency, liability, and ethical considerations. However, based on general knowledge, these key areas are often addressed in AI regulations. Data privacy regulations aim to protect personal data and ensure that AI systems handle data in a secure and responsible manner. Algorithmic transparency refers to the requirement for AI systems to provide explanations or justifications for their decisions or actions. Liability regulations determine who is responsible if an AI system causes harm or makes a mistake. Ethical considerations in AI regulations involve promoting fairness, accountability, and avoiding bias in AI systems.

> Finished chain.
*****

Step: Include information on the key areas covered by the regulations, such as data privacy, algorithmic transparency, liability, and ethical considerations.

Response: The search 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-k9LQtZKllzwGlzidOA94QWrY on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

At first, the planning agent creates a plan for our query with multiple steps. Each step is a query that the action agent will be asked to give an answer to. Here are the identified steps.

    Research the current state of Artificial Intelligence (AI) regulations in various countries.
    Identify the key countries with significant AI regulations or ongoing discussions about AI regulations.
    Summarize the AI regulations or discussions in each identified country.
    Organize the information by country, providing an overview of each AI regulation.
    Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country.

Then, we see the executor agent searching for relevant documents with our Deep Lake tool and showing the top 3 retrieved chunks. After analyzing them, the agent decides to return the final answer to the question.

Let’s print the final response from the agent.

In [18]:
print(response)

European Union: The AI Act has been endorsed by the Internal Market Committee and the Civil Liberties Committee, aiming to ensure AI systems are safe, transparent, traceable, and non-discriminatory. United States: AI developers typically experiment first and then retrofit their products to other markets and regulatory frameworks, fostering innovation. United Kingdom: The UK is adopting a similar approach to the US, but its proximity to the EU market means that UK-based developers are more likely to align with the EU ruleset from the outset. Regulatory sandboxes are being promoted to test AI before deployment.


We see that the agent has been able to iteratively create an overview of AI regulations by diverse documents, leveraging several documents.

The experiment involving the Plan and Execute agent has been successful in providing a comprehensive overview of Artificial Intelligence regulations by governments, specifically by finding information about the European Union, United States, and United Kingdom. The agent effectively performed various steps, including researching the current state of AI regulations, identifying key countries, summarizing regulations, and organizing the information by country.

The output generated by the agent demonstrates its ability to understand and interpret complex information about AI regulations. It accurately summarizes the AI regulations in each country, highlighting the endorsement of the AI Act in the European Union to ensure the safety, transparency, traceability, and non-discrimination of AI systems.

The agent successfully executes its plan by retrieving relevant information, summarizing it, and providing a concise and informative overview. It demonstrates its capability to gather insights from multiple sources and deliver a coherent response. The agent's performance in this experiment highlights its potential to assist with research, generate informative summaries, and provide valuable insights.